In [1]:
# ============================================================
# FULL MODIFIED CODE: Parameter-Efficient Custom CNN + Mixup DG
# (Train: ODIR + RFMiD_v2, Test: RFMiD_v1)
# ============================================================

import os
import random
from typing import Dict

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, f1_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
from tqdm import tqdm


# =============================================================================
# (A) HARMONIZATION UTILITIES
# =============================================================================

def harmonize_odir(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/odir-clr/ODIR_CLR"

    if split == "train":
        img_dir = f"{base_path}/Training_ Set/train_images"
        label_file = f"{base_path}/Training_ Set/train_annotation.xlsx"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set/val_images"
        label_file = f"{base_path}/Validation_set/val_annotation.xlsx"
    else:
        img_dir = f"{base_path}/Test_Set/test_images"
        label_file = f"{base_path}/Test_Set/test_annotation.xlsx"

    df = pd.read_excel(label_file)
    label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    harmonized_rows = []

    for _, row in df.iterrows():
        patient_id = row['ID']
        labels = {col: int(row[col]) for col in label_cols}

        if pd.notna(row.get('Left-Fundus')) and str(row['Left-Fundus']).strip():
            left_path = os.path.join(img_dir, f"{patient_id}_left.jpg")
            harmonized_rows.append({
                'image_path': left_path,
                'dataset': 'ODIR',
                'split': split,
                'ID': patient_id,
                **labels
            })

        if pd.notna(row.get('Right-Fundus')) and str(row['Right-Fundus']).strip():
            right_path = os.path.join(img_dir, f"{patient_id}_right.jpg")
            harmonized_rows.append({
                'image_path': right_path,
                'dataset': 'ODIR',
                'split': split,
                'ID': patient_id,
                **labels
            })

    return pd.DataFrame(harmonized_rows)


def harmonize_rfmid_v1(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/retinal-disease-classification"

    if split == "train":
        img_dir = f"{base_path}/Training_Set/Training_Set/Training"
        label_file = f"{base_path}/Training_Set/Training_Set/RFMiD_Training_Labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Evaluation_Set/Evaluation_Set/Validation"
        label_file = f"{base_path}/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv"
    else:
        img_dir = f"{base_path}/Test_Set/Test_Set/Test"
        label_file = f"{base_path}/Test_Set/Test_Set/RFMiD_Testing_Labels.csv"

    df = pd.read_csv(label_file)

    all_disease_cols = ['DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN',
                        'ERM', 'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH',
                        'ODP', 'ODE', 'ST', 'AION', 'PT', 'RT', 'RS', 'CRS',
                        'EDN', 'RPEC', 'MHL', 'RP', 'CWS', 'CB', 'ODPM',
                        'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
                        'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL']

    harmonized_rows = []

    for _, row in df.iterrows():
        image_id = row['ID']

        image_path = None
        for ext in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG']:
            candidate = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(candidate):
                image_path = candidate
                break
        if image_path is None:
            image_path = os.path.join(img_dir, str(image_id))

        # Direct single-column mapping
        N = 1 if row['Disease_Risk'] == 0 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0
        H = 1 if row.get('HR', 0) == 1 else 0
        M = 1 if row.get('MYA', 0) == 1 else 0

        used_for_mapping = ['DR', 'ODC', 'MH', 'ARMD', 'HR', 'MYA']
        other_cols = [col for col in all_disease_cols if col not in used_for_mapping]
        O = 1 if any(row.get(col, 0) == 1 for col in other_cols) else 0

        harmonized_rows.append({
            'image_path': image_path,
            'dataset': 'RFMiD_v1',
            'split': split,
            'ID': image_id,
            'N': N, 'D': D, 'G': G, 'C': C, 'A': A, 'H': H, 'M': M, 'O': O
        })

    return pd.DataFrame(harmonized_rows)


def harmonize_rfmid_v2(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/rdc-version-2/RFDiM2_0"

    if split == "train":
        img_dir = f"{base_path}/Training_set_2/Train_2"
        label_file = f"{base_path}/Training_set_2/RFMiD_2_Training_labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set_2/Validation_2"
        label_file = f"{base_path}/Validation_set_2/RFMiD_2_Validation_labels.csv"
    else:
        img_dir = f"{base_path}/Test_set_2/Test_2"
        label_file = f"{base_path}/Test_set_2/RFMiD_2_Testing_labels.csv"

    try:
        df = pd.read_csv(label_file, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(label_file, encoding='latin1')

    df.columns = df.columns.str.strip()

    potential_disease_cols = ['AH', 'AION', 'ARMD', 'BRVO', 'CB', 'CF', 'CL', 'CME',
                              'CNV', 'CRAO', 'CRS', 'CRVO', 'CSR', 'CWS', 'CSC', 'DN',
                              'DR', 'EDN', 'ERM', 'GRT', 'HPED', 'HR', 'LS', 'MCA',
                              'ME', 'MH', 'MHL', 'MS', 'MYA', 'ODC', 'ODE', 'ODP',
                              'ON', 'OPDM', 'PRH', 'RD', 'RHL', 'RTR', 'RP', 'RPEC',
                              'RS', 'RT', 'SOFE', 'ST', 'TD', 'TSLN', 'TV', 'VS',
                              'HTN', 'IIH']

    all_disease_cols = [col for col in potential_disease_cols if col in df.columns]
    harmonized_rows = []

    skipped_count = 0
    found_count = 0

    for _, row in df.iterrows():
        image_id = int(row['ID'])

        image_path = None
        for ext in ['.jpg', '.JPG', '.png', '.PNG', '.jpeg', '.JPEG']:
            candidate = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(candidate):
                image_path = candidate
                found_count += 1
                break

        if image_path is None:
            skipped_count += 1
            continue

        # Map labels (v2 uses WNL for Normal)
        wnl = row.get('WNL', 0)
        N = 1 if wnl == 1 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0

        # H: prefer HTN if present else HR fallback
        H = 1 if row.get('HTN', 0) == 1 else 0
        if H == 0 and 'HR' in df.columns:
            H = 1 if row.get('HR', 0) == 1 else 0

        M = 1 if row.get('MYA', 0) == 1 else 0

        used_for_mapping = ['DR', 'ODC', 'MH', 'ARMD', 'HTN', 'HR', 'MYA', 'WNL']
        other_cols = [col for col in all_disease_cols if col not in used_for_mapping]
        O = 1 if any(row.get(col, 0) == 1 for col in other_cols) else 0

        harmonized_rows.append({
            'image_path': image_path,
            'dataset': 'RFMiD_v2',
            'split': split,
            'ID': image_id,
            'N': N, 'D': D, 'G': G, 'C': C, 'A': A, 'H': H, 'M': M, 'O': O
        })

    if skipped_count > 0:
        print(f"RFMiD_v2 {split}: Found {found_count}, Skipped {skipped_count}")

    return pd.DataFrame(harmonized_rows)


def harmonize_all_datasets() -> Dict[str, pd.DataFrame]:
    results = {}

    print("\n" + "=" * 60)
    print("HARMONIZING DATASETS")
    print("=" * 60)

    for split in ['train', 'val', 'test']:
        print(f"\nProcessing {split} split...")
        results[f'ODIR_{split}'] = harmonize_odir(split)
        results[f'RFMiD_v1_{split}'] = harmonize_rfmid_v1(split)
        results[f'RFMiD_v2_{split}'] = harmonize_rfmid_v2(split)

    return results


def print_statistics(harmonized_data: Dict[str, pd.DataFrame]):
    label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

    print("\n" + "=" * 60)
    print("DATASET STATISTICS")
    print("=" * 60)

    print("\n📊 Sample Counts:")
    print("-" * 40)
    for key, df in sorted(harmonized_data.items()):
        print(f"{key:20s}: {len(df):5d} images")

    total_train = sum(len(df) for k, df in harmonized_data.items() if 'train' in k)
    total_val = sum(len(df) for k, df in harmonized_data.items() if 'val' in k)
    total_test = sum(len(df) for k, df in harmonized_data.items() if 'test' in k)

    print("-" * 40)
    print(f"{'Total Train':20s}: {total_train:5d} images")
    print(f"{'Total Val':20s}: {total_val:5d} images")
    print(f"{'Total Test':20s}: {total_test:5d} images")
    print(f"{'Grand Total':20s}: {total_train + total_val + total_test:5d} images")

    for dataset_name in ['ODIR', 'RFMiD_v1', 'RFMiD_v2']:
        dataset_dfs = {k: v for k, v in harmonized_data.items() if k.startswith(dataset_name)}
        if not dataset_dfs:
            continue

        print(f"\n{'=' * 60}")
        print(f"{dataset_name} Label Distribution")
        print(f"{'=' * 60}")

        for split in ['train', 'val', 'test']:
            key = f"{dataset_name}_{split}"
            if key not in dataset_dfs:
                continue

            df = dataset_dfs[key]
            print(f"\n{split.upper()} ({len(df)} images):")
            print("-" * 40)
            for col in label_cols:
                count = df[col].sum()
                pct = (count / len(df)) * 100 if len(df) > 0 else 0
                print(f"  {col}: {count:5d} ({pct:5.1f}%)")


def save_harmonized_data(harmonized_data: Dict[str, pd.DataFrame], output_dir: str = './harmonized_labels'):
    os.makedirs(output_dir, exist_ok=True)

    print("\n" + "=" * 60)
    print("SAVING HARMONIZED DATA")
    print("=" * 60)

    for key, df in harmonized_data.items():
        output_path = os.path.join(output_dir, f"{key}.csv")
        df.to_csv(output_path, index=False)
        print(f"✅ Saved {key:20s}: {len(df):5d} rows → {output_path}")

    print("\n📦 Creating combined files...")
    for split in ['train', 'val', 'test']:
        split_dfs = [v for k, v in harmonized_data.items() if k.endswith(f'_{split}')]
        if split_dfs:
            combined = pd.concat(split_dfs, ignore_index=True)
            output_path = os.path.join(output_dir, f"combined_{split}.csv")
            combined.to_csv(output_path, index=False)
            print(f"✅ Saved combined_{split:5s}: {len(combined):5d} rows → {output_path}")

    print(f"\n✨ All files saved to: {output_dir}")


def verify_images(harmonized_data: Dict[str, pd.DataFrame]):
    """Verify that all image paths in the harmonized data actually exist"""
    print("\n" + "=" * 60)
    print("VERIFYING IMAGE PATHS")
    print("=" * 60)

    all_good = True
    for key, df in sorted(harmonized_data.items()):
        missing = df[~df['image_path'].apply(os.path.exists)]
        if len(missing) > 0:
            print(f"❌ {key:20s}: {len(missing)} missing images")
            all_good = False
        else:
            print(f"✅ {key:20s}: All {len(df)} images found")

    if all_good:
        print("\n🎉 All image paths verified successfully!")
    else:
        print("\n⚠️  Some images are missing - check the paths above")


# =============================================================================
# (B) CONFIG for FOLD (Swapped: Train=ODIR+RFMiD_v2, Test=RFMiD_v1)
# =============================================================================

SEED = 42
SAVE_DIR = './results_lodo_mixup/custom_cnn_fold_test_RFMiD_v1'
TEST_DOMAIN = "RFMiD_v1"
TRAIN_DOMAINS = "ODIR + RFMiD_v2"

MIXUP_ALPHA = 0.2  # Controls mixing strength

# Paths (SWAPPED ROLES)
TEST_CSV = '/kaggle/working/harmonized_labels/RFMiD_v1_test.csv'
TRAIN_CSVS = [
    '/kaggle/working/harmonized_labels/ODIR_train.csv',
    '/kaggle/working/harmonized_labels/RFMiD_v2_train.csv'
]
VAL_CSVS = [
    '/kaggle/working/harmonized_labels/ODIR_val.csv',
    '/kaggle/working/harmonized_labels/RFMiD_v2_val.csv'
]


# =============================================================================
# Reproducibility
# =============================================================================

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


# =============================================================================
# Dataset (with domain tracking for Mixup!)
# =============================================================================

class RetinalDataset(Dataset):
    def __init__(self, csv_path, domain_id, transform=None):
        self.data = pd.read_csv(csv_path)
        self.domain_id = domain_id
        self.transform = transform
        self.label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]

        dup = self.data.duplicated(subset=["image_path"], keep="first").sum()
        if dup > 0:
            print(f"[WARN] {dup} duplicates found in {csv_path}, keeping first")
        self.data = self.data.drop_duplicates(subset=["image_path"]).reset_index(drop=True)

        valid_mask = self.data["image_path"].apply(os.path.exists)
        missing = (~valid_mask).sum()
        if missing > 0:
            print(f"[WARN] Dropping {missing} missing images from {csv_path}")
        self.data = self.data.loc[valid_mask].reset_index(drop=True)
        print(f"[INFO] Domain {domain_id}: Loaded {len(self.data)} valid images from {os.path.basename(csv_path)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:
            image = Image.open(row["image_path"]).convert("RGB")
        except Exception:
            image = Image.new("RGB", (224, 224), color=(128, 128, 128))

        labels = torch.tensor(row[self.label_cols].values.astype("float32"))
        if self.transform:
            image = self.transform(image)
        return image, labels, self.domain_id


# =============================================================================
# Pos weight
# =============================================================================

def calculate_pos_weights(datasets, clip_min=0.5, clip_max=50.0):
    all_labels = []
    for ds in datasets:
        all_labels.append(ds.data[["N", "D", "G", "C", "A", "H", "M", "O"]].values)
    combined = np.vstack(all_labels)

    pos = combined.sum(axis=0)
    neg = len(combined) - pos
    raw = neg / (pos + 1e-5)
    clipped = np.clip(raw, clip_min, clip_max)

    print("\n[INFO] Positive class weights (from training domains):")
    for i, col in enumerate(["N", "D", "G", "C", "A", "H", "M", "O"]):
        print(f"  {col}: {clipped[i]:.2f}")

    return torch.tensor(clipped, dtype=torch.float32)


# =============================================================================
# (C) PARAMETER-EFFICIENT CUSTOM CNN (Depthwise-Separable)
# =============================================================================

class SEBlock(nn.Module):
    """Squeeze-and-Excitation block (very small overhead, often helps)."""
    def __init__(self, channels: int, reduction: int = 8):
        super().__init__()
        hidden = max(4, channels // reduction)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, hidden),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.shape
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


class DepthwiseSeparableConv(nn.Module):
    """
    Depthwise Separable Conv:
      - depthwise: groups=in_ch
      - pointwise: 1x1
    Much fewer params than standard conv.
    """
    def __init__(self, in_ch: int, out_ch: int, stride: int = 1, use_se: bool = True):
        super().__init__()
        self.dw = nn.Conv2d(in_ch, in_ch, kernel_size=3, stride=stride, padding=1,
                            groups=in_ch, bias=False)
        self.dw_bn = nn.BatchNorm2d(in_ch)
        self.pw = nn.Conv2d(in_ch, out_ch, kernel_size=1, bias=False)
        self.pw_bn = nn.BatchNorm2d(out_ch)
        self.act = nn.SiLU(inplace=True)  # efficient + strong
        self.se = SEBlock(out_ch) if use_se else nn.Identity()

    def forward(self, x):
        x = self.act(self.dw_bn(self.dw(x)))
        x = self.act(self.pw_bn(self.pw(x)))
        x = self.se(x)
        return x


class TinyRetinaCNN(nn.Module):
    """
    Parameter-efficient CNN backbone + classifier head.
    Designed to be much smaller than ResNet50 but still strong.
    """
    def __init__(self, num_classes: int = 8, width_mult: float = 1.0, dropout: float = 0.3):
        super().__init__()

        def c(ch):  # channel scaling
            return max(8, int(ch * width_mult))

        self.stem = nn.Sequential(
            nn.Conv2d(3, c(32), kernel_size=3, stride=2, padding=1, bias=False),  # 224->112
            nn.BatchNorm2d(c(32)),
            nn.SiLU(inplace=True),
        )

        # Stages: progressively downsample to get global semantics
        self.stage1 = nn.Sequential(
            DepthwiseSeparableConv(c(32), c(48), stride=1, use_se=True),
            DepthwiseSeparableConv(c(48), c(64), stride=2, use_se=True),   # 112->56
        )
        self.stage2 = nn.Sequential(
            DepthwiseSeparableConv(c(64), c(96), stride=1, use_se=True),
            DepthwiseSeparableConv(c(96), c(128), stride=2, use_se=True),  # 56->28
        )
        self.stage3 = nn.Sequential(
            DepthwiseSeparableConv(c(128), c(160), stride=1, use_se=True),
            DepthwiseSeparableConv(c(160), c(192), stride=2, use_se=True), # 28->14
        )
        self.stage4 = nn.Sequential(
            DepthwiseSeparableConv(c(192), c(256), stride=2, use_se=True), # 14->7
            DepthwiseSeparableConv(c(256), c(256), stride=1, use_se=True),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)

        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(c(256), c(256)),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(c(256)),
            nn.Dropout(dropout),
            nn.Linear(c(256), num_classes)
        )

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.pool(x).flatten(1)
        return self.classifier(x)


# =============================================================================
# Transforms
# =============================================================================

def get_transforms(is_train=False):
    if is_train:
        return transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])


# =============================================================================
# Metrics
# =============================================================================

def compute_metrics(labels, probs, thresholds=None):
    n_classes = labels.shape[1]

    aucs = []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            aucs.append(roc_auc_score(labels[:, i], probs[:, i]))
        else:
            aucs.append(np.nan)

    aps = []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            aps.append(average_precision_score(labels[:, i], probs[:, i]))
        else:
            aps.append(np.nan)

    if thresholds is None:
        thresholds = np.full(n_classes, 0.5)

    preds = (probs >= thresholds).astype(int)
    f1 = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "mAUC": float(np.nanmean(aucs)),
        "mAP": float(np.nanmean(aps)),
        "per_class_auc": aucs,
        "per_class_ap": aps,
        "macro_f1": float(f1),
    }


def find_optimal_thresholds(labels, probs):
    n_classes = labels.shape[1]
    thresholds = []
    search_range = np.linspace(0.05, 0.95, 91)

    for i in range(n_classes):
        best_f1, best_t = 0.0, 0.5
        if len(np.unique(labels[:, i])) > 1:
            for t in search_range:
                preds = (probs[:, i] >= t).astype(int)
                f1 = f1_score(labels[:, i], preds, zero_division=0)
                if f1 > best_f1:
                    best_f1, best_t = f1, t
        thresholds.append(best_t)

    return np.array(thresholds)


# =============================================================================
# MIXUP TRAINING (inter-domain)
# =============================================================================

def train_epoch_mixup(model, loaders_dict, criterion, optimizer, device, mixup_alpha=0.2):
    model.train()
    losses = []
    mixup_stats = {'total_batches': 0, 'mixup_batches': 0}

    domain_iters = {k: iter(v) for k, v in loaders_dict.items()}
    domain_ids = list(loaders_dict.keys())

    max_batches = max(len(loader) for loader in loaders_dict.values())
    pbar = tqdm(range(max_batches), desc="Train (Mixup)", leave=False)

    for _ in pbar:
        if len(domain_ids) >= 2 and np.random.rand() > 0.5:
            d1, d2 = np.random.choice(domain_ids, size=2, replace=False)

            try:
                imgs1, labels1, _ = next(domain_iters[d1])
            except StopIteration:
                domain_iters[d1] = iter(loaders_dict[d1])
                imgs1, labels1, _ = next(domain_iters[d1])

            try:
                imgs2, labels2, _ = next(domain_iters[d2])
            except StopIteration:
                domain_iters[d2] = iter(loaders_dict[d2])
                imgs2, labels2, _ = next(domain_iters[d2])

            min_size = min(imgs1.size(0), imgs2.size(0))
            imgs1, labels1 = imgs1[:min_size], labels1[:min_size]
            imgs2, labels2 = imgs2[:min_size], labels2[:min_size]

            lam = np.random.beta(mixup_alpha, mixup_alpha)

            mixed_imgs = lam * imgs1 + (1 - lam) * imgs2
            mixed_labels = lam * labels1 + (1 - lam) * labels2

            mixed_imgs = mixed_imgs.to(device)
            mixed_labels = mixed_labels.to(device)

            optimizer.zero_grad(set_to_none=True)
            logits = model(mixed_imgs)
            loss = criterion(logits, mixed_labels)

            mixup_stats['mixup_batches'] += 1

        else:
            d = np.random.choice(domain_ids)
            try:
                imgs, labels, _ = next(domain_iters[d])
            except StopIteration:
                domain_iters[d] = iter(loaders_dict[d])
                imgs, labels, _ = next(domain_iters[d])

            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad(set_to_none=True)
            logits = model(imgs)
            loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        losses.append(loss.item())
        mixup_stats['total_batches'] += 1
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})

    return {
        "loss": float(np.mean(losses)),
        "mixup_ratio": mixup_stats['mixup_batches'] / max(1, mixup_stats['total_batches'])
    }


@torch.no_grad()
def validate(model, loader, criterion, device, thresholds=None):
    model.eval()
    losses, all_probs, all_labels = [], [], []

    for batch in tqdm(loader, desc="Val", leave=False):
        if len(batch) == 3:
            images, labels, _ = batch
        else:
            images, labels = batch

        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        losses.append(loss.item())
        all_probs.append(torch.sigmoid(logits).cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    probs = np.vstack(all_probs)
    labels = np.vstack(all_labels)
    metrics = compute_metrics(labels, probs, thresholds)
    metrics["loss"] = float(np.mean(losses))
    return metrics, probs, labels


# =============================================================================
# Visualizations
# =============================================================================

def plot_training_curves(history, save_dir):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    epochs = range(1, len(history['train_loss']) + 1)

    axes[0].plot(epochs, history['train_loss'], linewidth=2, label='Train Loss', marker='o')
    axes[0].plot(epochs, history['val_loss'], linewidth=2, label='Val Loss', marker='s')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss (Mixup)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(epochs, history['val_auc'], linewidth=2, label='Val mAUC', marker='s')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('mAUC')
    axes[1].set_title('Validation mAUC (Mixup)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0, 1.0])

    plt.tight_layout()
    plt.savefig(f'{save_dir}/training_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved training curves")


def plot_per_class_roc(labels, probs, test_domain, save_dir):
    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    n_classes = 8

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()

    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            fpr, tpr, _ = roc_curve(labels[:, i], probs[:, i])
            auc = roc_auc_score(labels[:, i], probs[:, i])

            axes[i].plot(fpr, tpr, linewidth=2, label=f'AUC = {auc:.3f}')
            axes[i].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
            axes[i].set_xlabel('False Positive Rate')
            axes[i].set_ylabel('True Positive Rate')
            axes[i].set_title(f'Class {class_names[i]}')
            axes[i].legend(loc='lower right')
            axes[i].grid(True, alpha=0.3)
        else:
            axes[i].text(0.5, 0.5, 'Single class\n(No ROC)', ha='center', va='center', fontsize=12)
            axes[i].set_title(f'Class {class_names[i]}')

    plt.suptitle(f'Per-Class ROC Curves - Test on {test_domain} (Mixup)', y=0.995)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/per_class_roc_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved per-class ROC curves")


def plot_macro_roc(labels, probs, test_domain, save_dir):
    n_classes = 8
    fig, ax = plt.subplots(figsize=(8, 6))

    all_fpr, all_tpr = [], []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            fpr, tpr, _ = roc_curve(labels[:, i], probs[:, i])
            all_fpr.append(fpr)
            all_tpr.append(tpr)

    mean_fpr = np.linspace(0, 1, 100)
    tprs = [np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)]
    mean_tpr = np.mean(tprs, axis=0)
    macro_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, linewidth=3, label=f'Macro-avg ROC (AUC = {macro_auc:.3f})')
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.5, label='Random')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'Macro-Average ROC Curve - Test on {test_domain} (Mixup)')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/macro_roc_curve.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved macro-average ROC curve")


def plot_per_class_metrics(test_metrics, save_dir):
    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    aucs = test_metrics['per_class_auc']
    aps = test_metrics['per_class_ap']

    aucs = [auc if not np.isnan(auc) else 0 for auc in aucs]
    aps = [ap if not np.isnan(ap) else 0 for ap in aps]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    x = np.arange(len(class_names))

    axes[0].bar(x, aucs, alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[0].set_xlabel('Disease Class')
    axes[0].set_ylabel('AUC')
    axes[0].set_title('Per-Class AUC (Mixup)')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(class_names)
    axes[0].set_ylim([0, 1.0])
    axes[0].grid(True, alpha=0.3, axis='y')
    for i, auc in enumerate(aucs):
        axes[0].text(i, auc + 0.02, f'{auc:.2f}', ha='center', fontsize=9)

    axes[1].bar(x, aps, alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[1].set_xlabel('Disease Class')
    axes[1].set_ylabel('Average Precision')
    axes[1].set_title('Per-Class Average Precision (Mixup)')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(class_names)
    axes[1].set_ylim([0, 1.0])
    axes[1].grid(True, alpha=0.3, axis='y')
    for i, ap in enumerate(aps):
        axes[1].text(i, ap + 0.02, f'{ap:.2f}', ha='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/per_class_metrics.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved per-class metrics chart")


# =============================================================================
# (D) MAIN TRAINING SCRIPT
# =============================================================================

def count_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


if __name__ == "__main__":
    # ---- (1) Harmonize + Save (optional: comment out if already done)
    harmonized_data = harmonize_all_datasets()
    print_statistics(harmonized_data)
    verify_images(harmonized_data)
    save_harmonized_data(harmonized_data, output_dir='./harmonized_labels')

    # ---- (2) Train / Val / Test
    set_seed(SEED)
    os.makedirs(SAVE_DIR, exist_ok=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print("=" * 80)
    print(f"LODO FOLD (SWAPPED) with MIXUP DG: Test on {TEST_DOMAIN}")
    print(f"Training on: {TRAIN_DOMAINS}")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Seed: {SEED}")
    print(f"Mixup Alpha: {MIXUP_ALPHA}")
    print("=" * 80)

    print("\n[INFO] Loading datasets...")
    train_datasets = [RetinalDataset(csv, domain_id=i, transform=get_transforms(True))
                      for i, csv in enumerate(TRAIN_CSVS)]
    val_datasets = [RetinalDataset(csv, domain_id=i, transform=get_transforms(False))
                    for i, csv in enumerate(VAL_CSVS)]
    test_dataset = RetinalDataset(TEST_CSV, domain_id=999, transform=get_transforms(False))

    print(f"\n[INFO] Total train: {sum(len(ds) for ds in train_datasets)} images")
    print(f"[INFO] Total val:   {sum(len(ds) for ds in val_datasets)} images")
    print(f"[INFO] Test:        {len(test_dataset)} images")

    pos_weights = calculate_pos_weights(train_datasets).to(device)

    g = torch.Generator().manual_seed(SEED)
    train_loaders = {
        i: DataLoader(ds, batch_size=32, shuffle=True, num_workers=4, pin_memory=True, generator=g)
        for i, ds in enumerate(train_datasets)
    }

    combined_val = ConcatDataset(val_datasets)
    val_loader = DataLoader(combined_val, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    # ---- Custom CNN model (parameter-efficient)
    print("\n[INFO] Initializing Parameter-Efficient Custom CNN...")
    model = TinyRetinaCNN(num_classes=8, width_mult=1.0, dropout=0.3).to(device)
    print(f"[INFO] Trainable params: {count_parameters(model):,}")

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

    # Single LR optimizer (no backbone/head split needed now)
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3
    )

    history = {
        'train_loss': [],
        'val_loss': [],
        'val_auc': []
    }

    best_val_auc, patience_counter = 0.0, 0
    print("\n[INFO] Training started with Mixup DG...")
    print("-" * 80)

    for epoch in range(50):
        train_metrics = train_epoch_mixup(
            model, train_loaders, criterion, optimizer, device, mixup_alpha=MIXUP_ALPHA
        )
        val_metrics, _, _ = validate(model, val_loader, criterion, device)
        scheduler.step(val_metrics['mAUC'])

        history['train_loss'].append(train_metrics['loss'])
        history['val_loss'].append(val_metrics['loss'])
        history['val_auc'].append(val_metrics['mAUC'])

        print(f"Epoch {epoch + 1:02d} | Train Loss: {train_metrics['loss']:.4f} "
              f"| Val mAUC: {val_metrics['mAUC']:.4f} | Mixup: {train_metrics['mixup_ratio']:.1%}")

        if val_metrics['mAUC'] > best_val_auc:
            best_val_auc = val_metrics['mAUC']
            torch.save(model.state_dict(), f'{SAVE_DIR}/best_model.pth')
            print(f"  ✓ Saved best model (val mAUC: {best_val_auc:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= 10:
            print(f"\n[INFO] Early stopping at epoch {epoch + 1}")
            break

    print(f"\n[INFO] Loading best model...")
    model.load_state_dict(torch.load(f'{SAVE_DIR}/best_model.pth', map_location=device))
    print(f"[INFO] Best validation mAUC: {best_val_auc:.4f}")

    print(f"\n[INFO] Finding optimal thresholds on validation...")
    _, val_probs, val_labels = validate(model, val_loader, criterion, device)
    thresholds = find_optimal_thresholds(val_labels, val_probs)

    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    print("[INFO] Optimal thresholds:")
    for c, t in zip(class_names, thresholds):
        print(f"  {c}: {t:.3f}")

    print(f"\n[INFO] Testing on {TEST_DOMAIN}...")
    print("-" * 80)
    test_metrics, test_probs, test_labels = validate(model, test_loader, criterion, device, thresholds)

    print("\n" + "=" * 80)
    print(f"TEST RESULTS - {TEST_DOMAIN} (Mixup DG) - Custom CNN")
    print("=" * 80)
    print(f"mAUC:      {test_metrics['mAUC']:.4f}")
    print(f"mAP:       {test_metrics['mAP']:.4f}")
    print(f"Macro F1:  {test_metrics['macro_f1']:.4f}")
    print("=" * 80)

    print(f"\n{'Class':<8} {'AUC':<10} {'AP':<10}")
    print("-" * 30)
    for i, cls in enumerate(class_names):
        auc = test_metrics['per_class_auc'][i]
        ap = test_metrics['per_class_ap'][i]
        auc_str = f"{auc:.4f}" if not np.isnan(auc) else "N/A"
        ap_str = f"{ap:.4f}" if not np.isnan(ap) else "N/A"
        print(f"{cls:<8} {auc_str:<10} {ap_str:<10}")
    print("-" * 30)

    print("\n[INFO] Saving results...")
    results_df = pd.DataFrame([{
        'method': 'Mixup + CustomCNN',
        'test_domain': TEST_DOMAIN,
        'train_domains': TRAIN_DOMAINS,
        'mAUC': test_metrics['mAUC'],
        'mAP': test_metrics['mAP'],
        'macro_f1': test_metrics['macro_f1'],
        'best_val_auc': best_val_auc,
        'mixup_alpha': MIXUP_ALPHA,
        'trainable_params': count_parameters(model)
    }])
    results_df.to_csv(f'{SAVE_DIR}/test_results.csv', index=False)
    print(f"[INFO] ✓ Saved test_results.csv")

    print("\n[INFO] Generating visualizations...")
    plot_training_curves(history, SAVE_DIR)
    plot_per_class_roc(test_labels, test_probs, TEST_DOMAIN, SAVE_DIR)
    plot_macro_roc(test_labels, test_probs, TEST_DOMAIN, SAVE_DIR)
    plot_per_class_metrics(test_metrics, SAVE_DIR)

    print("\n" + "=" * 80)
    print(f"✓ MIXUP LODO FOLD (SWAPPED) COMPLETE!")
    print(f"✓ Results saved to: {SAVE_DIR}/")
    print("=" * 80)



HARMONIZING DATASETS

Processing train split...
RFMiD_v2 train: Found 507, Skipped 2

Processing val split...

Processing test split...
RFMiD_v2 test: Found 170, Skipped 4

DATASET STATISTICS

📊 Sample Counts:
----------------------------------------
ODIR_test           :  2000 images
ODIR_train          :  7000 images
ODIR_val            :  1000 images
RFMiD_v1_test       :   640 images
RFMiD_v1_train      :  1920 images
RFMiD_v1_val        :   640 images
RFMiD_v2_test       :   170 images
RFMiD_v2_train      :   507 images
RFMiD_v2_val        :   177 images
----------------------------------------
Total Train         :  9427 images
Total Val           :  1817 images
Total Test          :  2810 images
Grand Total         : 14054 images

ODIR Label Distribution

TRAIN (7000 images):
----------------------------------------
  N:  2280 ( 32.6%)
  D:  2256 ( 32.2%)
  G:   430 (  6.1%)
  C:   424 (  6.1%)
  A:   328 (  4.7%)
  H:   206 (  2.9%)
  M:   348 (  5.0%)
  O:  1958 ( 28.0%)

VAL

Epoch 01 | Train Loss: 1.3703 | Val mAUC: 0.6103 | Mixup: 53.9%
  ✓ Saved best model (val mAUC: 0.6103)


Epoch 02 | Train Loss: 1.1782 | Val mAUC: 0.5911 | Mixup: 44.3%


Epoch 03 | Train Loss: 1.0945 | Val mAUC: 0.6272 | Mixup: 51.6%
  ✓ Saved best model (val mAUC: 0.6272)


Epoch 04 | Train Loss: 1.0789 | Val mAUC: 0.6524 | Mixup: 48.4%
  ✓ Saved best model (val mAUC: 0.6524)


...

Epoch 50 | Train Loss: 0.6857 | Val mAUC: 0.7253 | Mixup: 49.8%

[INFO] Loading best model...
[INFO] Best validation mAUC: 0.7312

[INFO] Finding optimal thresholds on validation...


[INFO] Optimal thresholds:
  N: 0.260
  D: 0.290
  G: 0.370
  C: 0.840
  A: 0.440
  H: 0.480
  M: 0.600
  O: 0.480

[INFO] Testing on RFMiD_v1...
--------------------------------------------------------------------------------



TEST RESULTS - RFMiD_v1 (Mixup DG) - Custom CNN
mAUC:      0.7778
mAP:       0.4223
Macro F1:  0.3709

Class    AUC        AP        
------------------------------
N        0.8740     0.7033    
D        0.7304     0.4344    
G        0.6281     0.2272    
C        0.8890     0.7086    
A        0.7655     0.2036    
H        0.7449     0.0061    
M        0.9421     0.5878    
O        0.6479     0.5077    
------------------------------

[INFO] Saving results...
[INFO] ✓ Saved test_results.csv

[INFO] Generating visualizations...
[INFO] ✓ Saved training curves
[INFO] ✓ Saved per-class ROC curves


/tmp/ipykernel_55/395585924.py:791: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  macro_auc = np.trapz(mean_tpr, mean_fpr)


[INFO] ✓ Saved macro-average ROC curve
[INFO] ✓ Saved per-class metrics chart

✓ MIXUP LODO FOLD (SWAPPED) COMPLETE!
✓ Results saved to: ./results_lodo_mixup/custom_cnn_fold_test_RFMiD_v1/


In [1]:
# ============================================================
# FAST + STRONG RetinaEffCNN (same architecture) + Mixup/CutMix + EMA + ASL
# Train: ODIR + RFMiD_v2 | Test: RFMiD_v1
# ============================================================

import os
import random
from typing import Dict, Optional, Tuple

import numpy as np
import pandas as pd
from PIL import Image, ImageFile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score, f1_score, average_precision_score
from tqdm import tqdm
import matplotlib.pyplot as plt

# PIL safety (helps avoid rare crashes)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# =============================================================================
# (A) HARMONIZATION UTILITIES (same logic as your working version)
# =============================================================================

def harmonize_odir(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/odir-clr/ODIR_CLR"
    if split == "train":
        img_dir = f"{base_path}/Training_ Set/train_images"
        label_file = f"{base_path}/Training_ Set/train_annotation.xlsx"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set/val_images"
        label_file = f"{base_path}/Validation_set/val_annotation.xlsx"
    else:
        img_dir = f"{base_path}/Test_Set/test_images"
        label_file = f"{base_path}/Test_Set/test_annotation.xlsx"

    df = pd.read_excel(label_file)
    label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    rows = []

    for _, row in df.iterrows():
        pid = row['ID']
        labels = {c: int(row[c]) for c in label_cols}

        if pd.notna(row.get('Left-Fundus')) and str(row['Left-Fundus']).strip():
            rows.append({
                "image_path": os.path.join(img_dir, f"{pid}_left.jpg"),
                "dataset": "ODIR", "split": split, "ID": pid, **labels
            })

        if pd.notna(row.get('Right-Fundus')) and str(row['Right-Fundus']).strip():
            rows.append({
                "image_path": os.path.join(img_dir, f"{pid}_right.jpg"),
                "dataset": "ODIR", "split": split, "ID": pid, **labels
            })

    return pd.DataFrame(rows)


def harmonize_rfmid_v1(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/retinal-disease-classification"
    if split == "train":
        img_dir = f"{base_path}/Training_Set/Training_Set/Training"
        label_file = f"{base_path}/Training_Set/Training_Set/RFMiD_Training_Labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Evaluation_Set/Evaluation_Set/Validation"
        label_file = f"{base_path}/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv"
    else:
        img_dir = f"{base_path}/Test_Set/Test_Set/Test"
        label_file = f"{base_path}/Test_Set/Test_Set/RFMiD_Testing_Labels.csv"

    df = pd.read_csv(label_file)

    all_disease_cols = ['DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN',
                        'ERM', 'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH',
                        'ODP', 'ODE', 'ST', 'AION', 'PT', 'RT', 'RS', 'CRS',
                        'EDN', 'RPEC', 'MHL', 'RP', 'CWS', 'CB', 'ODPM',
                        'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
                        'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL']

    rows = []
    for _, row in df.iterrows():
        image_id = row['ID']
        image_path = None
        for ext in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG']:
            cand = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(cand):
                image_path = cand
                break
        if image_path is None:
            image_path = os.path.join(img_dir, str(image_id))

        N = 1 if row['Disease_Risk'] == 0 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0
        H = 1 if row.get('HR', 0) == 1 else 0
        M = 1 if row.get('MYA', 0) == 1 else 0

        used = ['DR', 'ODC', 'MH', 'ARMD', 'HR', 'MYA']
        other_cols = [c for c in all_disease_cols if c not in used]
        O = 1 if any(row.get(c, 0) == 1 for c in other_cols) else 0

        rows.append({
            "image_path": image_path, "dataset": "RFMiD_v1", "split": split, "ID": image_id,
            "N": N, "D": D, "G": G, "C": C, "A": A, "H": H, "M": M, "O": O
        })

    return pd.DataFrame(rows)


def harmonize_rfmid_v2(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/rdc-version-2/RFDiM2_0"
    if split == "train":
        img_dir = f"{base_path}/Training_set_2/Train_2"
        label_file = f"{base_path}/Training_set_2/RFMiD_2_Training_labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set_2/Validation_2"
        label_file = f"{base_path}/Validation_set_2/RFMiD_2_Validation_labels.csv"
    else:
        img_dir = f"{base_path}/Test_set_2/Test_2"
        label_file = f"{base_path}/Test_set_2/RFMiD_2_Testing_labels.csv"

    try:
        df = pd.read_csv(label_file, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(label_file, encoding="latin1")

    df.columns = df.columns.str.strip()

    potential = ['AH', 'AION', 'ARMD', 'BRVO', 'CB', 'CF', 'CL', 'CME',
                 'CNV', 'CRAO', 'CRS', 'CRVO', 'CSR', 'CWS', 'CSC', 'DN',
                 'DR', 'EDN', 'ERM', 'GRT', 'HPED', 'HR', 'LS', 'MCA',
                 'ME', 'MH', 'MHL', 'MS', 'MYA', 'ODC', 'ODE', 'ODP',
                 'ON', 'OPDM', 'PRH', 'RD', 'RHL', 'RTR', 'RP', 'RPEC',
                 'RS', 'RT', 'SOFE', 'ST', 'TD', 'TSLN', 'TV', 'VS',
                 'HTN', 'IIH']

    all_disease_cols = [c for c in potential if c in df.columns]
    rows = []
    found, skipped = 0, 0

    for _, row in df.iterrows():
        image_id = int(row["ID"])
        image_path = None
        for ext in ['.jpg', '.JPG', '.png', '.PNG', '.jpeg', '.JPEG']:
            cand = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(cand):
                image_path = cand
                found += 1
                break
        if image_path is None:
            skipped += 1
            continue

        N = 1 if row.get("WNL", 0) == 1 else 0
        D = 1 if row.get("DR", 0) == 1 else 0
        G = 1 if row.get("ODC", 0) == 1 else 0
        C = 1 if row.get("MH", 0) == 1 else 0
        A = 1 if row.get("ARMD", 0) == 1 else 0

        H = 1 if row.get("HTN", 0) == 1 else 0
        if H == 0 and "HR" in df.columns:
            H = 1 if row.get("HR", 0) == 1 else 0

        M = 1 if row.get("MYA", 0) == 1 else 0

        used = ['DR', 'ODC', 'MH', 'ARMD', 'HTN', 'HR', 'MYA', 'WNL']
        other_cols = [c for c in all_disease_cols if c not in used]
        O = 1 if any(row.get(c, 0) == 1 for c in other_cols) else 0

        rows.append({
            "image_path": image_path, "dataset": "RFMiD_v2", "split": split, "ID": image_id,
            "N": N, "D": D, "G": G, "C": C, "A": A, "H": H, "M": M, "O": O
        })

    if skipped > 0:
        print(f"RFMiD_v2 {split}: Found {found}, Skipped {skipped}")

    return pd.DataFrame(rows)


def harmonize_all_datasets() -> Dict[str, pd.DataFrame]:
    out = {}
    print("\n" + "=" * 60)
    print("HARMONIZING DATASETS")
    print("=" * 60)
    for split in ["train", "val", "test"]:
        print(f"\nProcessing {split} split...")
        out[f"ODIR_{split}"] = harmonize_odir(split)
        out[f"RFMiD_v1_{split}"] = harmonize_rfmid_v1(split)
        out[f"RFMiD_v2_{split}"] = harmonize_rfmid_v2(split)
    return out


def save_harmonized_data(harmonized_data: Dict[str, pd.DataFrame], output_dir: str = "./harmonized_labels"):
    os.makedirs(output_dir, exist_ok=True)
    for k, df in harmonized_data.items():
        df.to_csv(os.path.join(output_dir, f"{k}.csv"), index=False)


# =============================================================================
# (B) CONFIG (FAST_MODE defaults ON)
# =============================================================================

FAST_MODE = True  # <- keep True for speed; set False for heavier training.

SEED = 42
SAVE_DIR = "./results_lodo_mixup/retinaeffcnn_fast"
TEST_DOMAIN = "RFMiD_v1"
TRAIN_DOMAINS = "ODIR + RFMiD_v2"

if FAST_MODE:
    IMG_SIZE = 288
    BATCH_SIZE = 32
    ACCUM_STEPS = 1
    EPOCHS = 35
    EARLY_STOP_PATIENCE = 8
    MIXUP_ALPHA = 0.20
    P_MIX = 0.55
    P_CUTMIX_IN_MIX = 0.45
    LABEL_SMOOTH = 0.01
    USE_TTA_ON_TEST = False
    WIDTH_MULT = 1.20
else:
    IMG_SIZE = 320
    BATCH_SIZE = 24
    ACCUM_STEPS = 2
    EPOCHS = 60
    EARLY_STOP_PATIENCE = 12
    MIXUP_ALPHA = 0.25
    P_MIX = 0.60
    P_CUTMIX_IN_MIX = 0.55
    LABEL_SMOOTH = 0.02
    USE_TTA_ON_TEST = True
    WIDTH_MULT = 1.35  # bigger but slower

EMA_DECAY = 0.999
USE_AMP = True

ASL_GAMMA_POS = 0.0
ASL_GAMMA_NEG = 2.0
ASL_CLIP = 0.05

TEST_CSV = "/kaggle/working/harmonized_labels/RFMiD_v1_test.csv"
TRAIN_CSVS = [
    "/kaggle/working/harmonized_labels/ODIR_train.csv",
    "/kaggle/working/harmonized_labels/RFMiD_v2_train.csv",
]
VAL_CSVS = [
    "/kaggle/working/harmonized_labels/ODIR_val.csv",
    "/kaggle/working/harmonized_labels/RFMiD_v2_val.csv",
]


# =============================================================================
# Reproducibility + speed knobs
# =============================================================================

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    os.environ["PYTHONHASHSEED"] = str(seed)
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass


# =============================================================================
# Dataset
# =============================================================================

class RetinalDataset(Dataset):
    def __init__(self, csv_path, domain_id, transform=None):
        self.data = pd.read_csv(csv_path)
        self.domain_id = domain_id
        self.transform = transform
        self.label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]

        self.data = self.data.drop_duplicates(subset=["image_path"]).reset_index(drop=True)
        valid = self.data["image_path"].apply(os.path.exists)
        self.data = self.data.loc[valid].reset_index(drop=True)

        print(f"[INFO] Domain {domain_id}: Loaded {len(self.data)} images from {os.path.basename(csv_path)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:
            img = Image.open(row["image_path"]).convert("RGB")
        except Exception:
            img = Image.new("RGB", (IMG_SIZE, IMG_SIZE), color=(128, 128, 128))

        y = torch.tensor(row[self.label_cols].values.astype("float32"))

        if self.transform:
            img = self.transform(img)
        return img, y, self.domain_id


def calculate_pos_weights(datasets, clip_min=0.5, clip_max=50.0):
    arr = []
    for ds in datasets:
        arr.append(ds.data[["N", "D", "G", "C", "A", "H", "M", "O"]].values)
    combined = np.vstack(arr)
    pos = combined.sum(axis=0)
    neg = len(combined) - pos
    w = neg / (pos + 1e-5)
    w = np.clip(w, clip_min, clip_max)

    print("\n[INFO] Positive class weights:")
    for i, c in enumerate(["N", "D", "G", "C", "A", "H", "M", "O"]):
        print(f"  {c}: {w[i]:.2f}")
    return torch.tensor(w, dtype=torch.float32)


# =============================================================================
# (C) SAME CNN ARCHITECTURE (RetinaEffCNN) - only width_mult configurable
# =============================================================================

class DropPath(nn.Module):
    def __init__(self, drop_prob: float = 0.0):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0.0 or (not self.training):
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rnd = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        rnd.floor_()
        return x.div(keep_prob) * rnd


class SEBlock(nn.Module):
    def __init__(self, channels: int, reduction: int = 4):
        super().__init__()
        hidden = max(8, channels // reduction)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, hidden)
        self.fc2 = nn.Linear(hidden, channels)

    def forward(self, x):
        b, c, _, _ = x.shape
        y = self.pool(x).view(b, c)
        y = torch.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(b, c, 1, 1)
        return x * y


class InvertedResidual(nn.Module):
    def __init__(self, in_ch, out_ch, stride, expand_ratio, use_se=True, drop_path=0.0):
        super().__init__()
        assert stride in [1, 2]
        self.use_res = (stride == 1 and in_ch == out_ch)
        hidden = int(round(in_ch * expand_ratio))

        if expand_ratio != 1:
            self.expand = nn.Sequential(
                nn.Conv2d(in_ch, hidden, 1, bias=False),
                nn.BatchNorm2d(hidden),
                nn.SiLU(inplace=True),
            )
        else:
            self.expand = nn.Identity()

        self.dw = nn.Sequential(
            nn.Conv2d(hidden, hidden, 3, stride=stride, padding=1, groups=hidden, bias=False),
            nn.BatchNorm2d(hidden),
            nn.SiLU(inplace=True),
        )

        self.se = SEBlock(hidden) if use_se else nn.Identity()

        self.project = nn.Sequential(
            nn.Conv2d(hidden, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch),
        )

        self.drop_path = DropPath(drop_path) if drop_path > 0 else nn.Identity()

    def forward(self, x):
        out = self.expand(x)
        out = self.dw(out)
        out = self.se(out)
        out = self.project(out)
        if self.use_res:
            out = x + self.drop_path(out)
        return out


class RetinaEffCNN(nn.Module):
    def __init__(self, num_classes=8, width_mult=1.20, dropout=0.35, drop_path_rate=0.10):
        super().__init__()

        def c(ch):
            return max(8, int(ch * width_mult))

        self.stem = nn.Sequential(
            nn.Conv2d(3, c(24), 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(c(24)),
            nn.SiLU(inplace=True),
        )

        cfg = [
            (c(24),  c(24),  1, 1.0, True),
            (c(24),  c(32),  2, 4.0, True),
            (c(32),  c(32),  1, 3.0, True),

            (c(32),  c(48),  2, 4.0, True),
            (c(48),  c(48),  1, 3.0, True),

            (c(48),  c(80),  2, 4.0, True),
            (c(80),  c(80),  1, 3.0, True),
            (c(80),  c(96),  1, 3.0, True),

            (c(96),  c(160), 2, 6.0, True),
            (c(160), c(160), 1, 6.0, True),
        ]

        blocks = []
        n = len(cfg)
        for i, (inch, outch, s, exp, se) in enumerate(cfg):
            dp = drop_path_rate * i / max(1, n - 1)
            blocks.append(InvertedResidual(inch, outch, s, exp, use_se=se, drop_path=dp))
        self.blocks = nn.Sequential(*blocks)

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(1),
            nn.Dropout(dropout),
            nn.Linear(c(160), c(256)),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(c(256)),
            nn.Dropout(dropout),
            nn.Linear(c(256), num_classes),
        )

        self._init()

    def _init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        return x


def count_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# =============================================================================
# (D) Asymmetric Loss
# =============================================================================

class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_pos=0.0, gamma_neg=2.0, clip=0.05, eps=1e-8, pos_weight=None):
        super().__init__()
        self.gp = gamma_pos
        self.gn = gamma_neg
        self.clip = clip
        self.eps = eps
        self.register_buffer("pos_weight", pos_weight if pos_weight is not None else None)

    def forward(self, logits, targets):
        targets = targets.clamp(0, 1)
        x = torch.sigmoid(logits)

        xs_pos = x
        xs_neg = 1.0 - x
        if self.clip and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1.0)

        loss_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        loss_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        loss = loss_pos + loss_neg

        if self.gp > 0 or self.gn > 0:
            pt = xs_pos * targets + xs_neg * (1 - targets)
            gamma = self.gp * targets + self.gn * (1 - targets)
            loss *= torch.pow(1 - pt, gamma)

        if self.pos_weight is not None:
            loss *= (targets * self.pos_weight + (1 - targets))

        return -loss.mean()


# =============================================================================
# Transforms (FASTER than your previous heavy pipeline)
# =============================================================================

def get_transforms(is_train=False):
    if is_train:
        # fast but still good
        return transforms.Compose([
            transforms.Resize((IMG_SIZE + 32, IMG_SIZE + 32)),
            transforms.RandomCrop(IMG_SIZE),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(8),
            transforms.ColorJitter(brightness=0.18, contrast=0.18, saturation=0.08, hue=0.02),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            transforms.RandomErasing(p=0.12, scale=(0.02, 0.10), ratio=(0.3, 3.3), value='random')
        ])
    else:
        return transforms.Compose([
            transforms.Resize((IMG_SIZE + 32, IMG_SIZE + 32)),
            transforms.CenterCrop(IMG_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])


# =============================================================================
# Metrics
# =============================================================================

def compute_metrics(labels, probs, thresholds=None):
    n = labels.shape[1]
    aucs, aps = [], []

    for i in range(n):
        if len(np.unique(labels[:, i])) > 1:
            aucs.append(roc_auc_score(labels[:, i], probs[:, i]))
            aps.append(average_precision_score(labels[:, i], probs[:, i]))
        else:
            aucs.append(np.nan)
            aps.append(np.nan)

    if thresholds is None:
        thresholds = np.full(n, 0.5)

    preds = (probs >= thresholds).astype(int)
    f1 = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "mAUC": float(np.nanmean(aucs)),
        "mAP": float(np.nanmean(aps)),
        "per_class_auc": aucs,
        "per_class_ap": aps,
        "macro_f1": float(f1),
    }


def find_optimal_thresholds(labels, probs):
    n = labels.shape[1]
    thresholds = []
    grid = np.linspace(0.05, 0.95, 91)
    for i in range(n):
        best_f1, best_t = 0.0, 0.5
        if len(np.unique(labels[:, i])) > 1:
            for t in grid:
                pred = (probs[:, i] >= t).astype(int)
                f1 = f1_score(labels[:, i], pred, zero_division=0)
                if f1 > best_f1:
                    best_f1, best_t = f1, t
        thresholds.append(best_t)
    return np.array(thresholds)


# =============================================================================
# Mixup/CutMix (inter-domain)
# =============================================================================

def smooth_labels(y: torch.Tensor, eps: float) -> torch.Tensor:
    if eps <= 0:
        return y
    return y * (1 - eps) + 0.5 * eps


def rand_bbox(W: int, H: int, lam: float) -> Tuple[int, int, int, int]:
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    x1 = np.clip(cx - cut_w // 2, 0, W)
    y1 = np.clip(cy - cut_h // 2, 0, H)
    x2 = np.clip(cx + cut_w // 2, 0, W)
    y2 = np.clip(cy + cut_h // 2, 0, H)
    return x1, y1, x2, y2


def interdomain_mix(imgs1, labels1, imgs2, labels2, alpha, p_cutmix=0.5):
    lam = np.random.beta(alpha, alpha)
    if np.random.rand() < p_cutmix:
        B, C, H, W = imgs1.shape
        x1, y1, x2, y2 = rand_bbox(W, H, lam)
        imgs = imgs1.clone()
        imgs[:, :, y1:y2, x1:x2] = imgs2[:, :, y1:y2, x1:x2]
        area = (x2 - x1) * (y2 - y1)
        lam_adj = 1.0 - area / float(W * H)
        labels = lam_adj * labels1 + (1 - lam_adj) * labels2
        return imgs, labels
    else:
        imgs = lam * imgs1 + (1 - lam) * imgs2
        labels = lam * labels1 + (1 - lam) * labels2
        return imgs, labels


# =============================================================================
# EMA
# =============================================================================

class ModelEMA:
    def __init__(self, model: nn.Module, decay: float = 0.999):
        self.decay = decay
        self.shadow = {n: p.detach().clone() for n, p in model.named_parameters() if p.requires_grad}

    @torch.no_grad()
    def update(self, model: nn.Module):
        for n, p in model.named_parameters():
            if n in self.shadow:
                self.shadow[n].mul_(self.decay).add_(p.detach(), alpha=1 - self.decay)

    def apply_to(self, model: nn.Module):
        self._backup = {}
        for n, p in model.named_parameters():
            if n in self.shadow:
                self._backup[n] = p.detach().clone()
                p.data.copy_(self.shadow[n].data)

    def restore(self, model: nn.Module):
        for n, p in model.named_parameters():
            if n in getattr(self, "_backup", {}):
                p.data.copy_(self._backup[n].data)
        self._backup = {}


# =============================================================================
# Train/Val (scheduler steps ONLY on optimizer update)
# =============================================================================

def train_epoch_mixup(
    model, loaders_dict, criterion, optimizer, device,
    scaler: Optional[torch.amp.GradScaler],
    ema: Optional[ModelEMA],
    scheduler: Optional[optim.lr_scheduler._LRScheduler],
    epoch_steps: int
):
    model.train()
    losses = []
    mix_ratio = 0

    iters = {k: iter(v) for k, v in loaders_dict.items()}
    domain_ids = list(loaders_dict.keys())

    optimizer.zero_grad(set_to_none=True)

    pbar = tqdm(range(epoch_steps), desc="Train", leave=False)
    for step in pbar:
        do_mix = (len(domain_ids) >= 2) and (np.random.rand() < P_MIX)

        if do_mix:
            d1, d2 = np.random.choice(domain_ids, size=2, replace=False)
            try:
                imgs1, labels1, _ = next(iters[d1])
            except StopIteration:
                iters[d1] = iter(loaders_dict[d1])
                imgs1, labels1, _ = next(iters[d1])

            try:
                imgs2, labels2, _ = next(iters[d2])
            except StopIteration:
                iters[d2] = iter(loaders_dict[d2])
                imgs2, labels2, _ = next(iters[d2])

            m = min(imgs1.size(0), imgs2.size(0))
            imgs1, labels1 = imgs1[:m], labels1[:m]
            imgs2, labels2 = imgs2[:m], labels2[:m]

            imgs, labels = interdomain_mix(imgs1, labels1, imgs2, labels2, MIXUP_ALPHA, P_CUTMIX_IN_MIX)
            mix_ratio += 1
        else:
            d = np.random.choice(domain_ids)  # uniform domain sampling
            try:
                imgs, labels, _ = next(iters[d])
            except StopIteration:
                iters[d] = iter(loaders_dict[d])
                imgs, labels, _ = next(iters[d])

        labels = smooth_labels(labels, LABEL_SMOOTH)

        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        if device.type == "cuda":
            imgs = imgs.contiguous(memory_format=torch.channels_last)

        with torch.amp.autocast(device_type="cuda", enabled=(USE_AMP and device.type == "cuda")):
            logits = model(imgs)
            loss = criterion(logits, labels)
            loss = loss / ACCUM_STEPS

        if scaler is not None and (USE_AMP and device.type == "cuda"):
            scaler.scale(loss).backward()
        else:
            loss.backward()

        if (step + 1) % ACCUM_STEPS == 0:
            if scaler is not None and (USE_AMP and device.type == "cuda"):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            optimizer.zero_grad(set_to_none=True)

            if scheduler is not None:
                scheduler.step()

            if ema is not None:
                ema.update(model)

        losses.append(loss.item() * ACCUM_STEPS)
        pbar.set_postfix({"loss": f"{np.mean(losses[-20:]):.4f}", "lr": f"{optimizer.param_groups[0]['lr']:.2e}"})

    return {
        "loss": float(np.mean(losses)),
        "mixup_ratio": float(mix_ratio / max(1, epoch_steps))
    }


@torch.no_grad()
def validate(model, loader, criterion, device, thresholds=None, tta=False):
    model.eval()
    losses, all_probs, all_labels = [], [], []

    for images, labels, _ in tqdm(loader, desc="Val", leave=False):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        if device.type == "cuda":
            images = images.contiguous(memory_format=torch.channels_last)

        logits = model(images)
        if tta:
            logits2 = model(torch.flip(images, dims=[3]))
            logits = (logits + logits2) / 2.0

        loss = criterion(logits, labels)
        losses.append(loss.item())

        probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.append(probs)
        all_labels.append(labels.cpu().numpy())

    probs = np.vstack(all_probs)
    labels = np.vstack(all_labels)
    metrics = compute_metrics(labels, probs, thresholds)
    metrics["loss"] = float(np.mean(losses))
    return metrics, probs, labels


def plot_training_curves(history, save_dir):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    ep = range(1, len(history["train_loss"]) + 1)

    axes[0].plot(ep, history["train_loss"], label="Train Loss")
    axes[0].plot(ep, history["val_loss"], label="Val Loss")
    axes[0].legend(); axes[0].grid(True, alpha=0.3)
    axes[0].set_title("Loss")

    axes[1].plot(ep, history["val_auc"], label="Val mAUC")
    axes[1].legend(); axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0, 1.0])
    axes[1].set_title("Val mAUC")

    os.makedirs(save_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(f"{save_dir}/training_curves.png", dpi=250)
    plt.close()


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    set_seed(SEED)
    os.makedirs(SAVE_DIR, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Harmonize if missing
    if not os.path.exists("/kaggle/working/harmonized_labels/ODIR_train.csv"):
        harmonized = harmonize_all_datasets()
        save_harmonized_data(harmonized, output_dir="./harmonized_labels")

    print("=" * 90)
    print(f"Mode: {'FAST' if FAST_MODE else 'HEAVY'} | IMG={IMG_SIZE} | BATCH={BATCH_SIZE} | ACCUM={ACCUM_STEPS} | AMP={USE_AMP}")
    print(f"Train: {TRAIN_DOMAINS} | Test: {TEST_DOMAIN}")
    print("=" * 90)

    train_datasets = [RetinalDataset(p, i, transform=get_transforms(True)) for i, p in enumerate(TRAIN_CSVS)]
    val_datasets = [RetinalDataset(p, i, transform=get_transforms(False)) for i, p in enumerate(VAL_CSVS)]
    test_dataset = RetinalDataset(TEST_CSV, 999, transform=get_transforms(False))

    pos_weights = calculate_pos_weights(train_datasets).to(device)

    # loaders
    g = torch.Generator().manual_seed(SEED)
    train_loaders = {
        i: DataLoader(
            ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True,
            num_workers=4, pin_memory=True,
            persistent_workers=True, prefetch_factor=2,
            generator=g
        )
        for i, ds in enumerate(train_datasets)
    }

    combined_val = ConcatDataset(val_datasets)
    val_loader = DataLoader(combined_val, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                             num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)

    # auto epoch steps (prevents wasting time)
    # ODIR batches ~292, RFMiD_v2 batches ~21 -> using max makes sense
    max_batches = max(len(dl) for dl in train_loaders.values())
    epoch_steps = max_batches  # fast + consistent
    print(f"[INFO] epoch_steps (auto) = {epoch_steps}")

    # model
    model = RetinaEffCNN(num_classes=8, width_mult=WIDTH_MULT, dropout=0.35, drop_path_rate=0.10).to(device)
    if device.type == "cuda":
        model = model.to(memory_format=torch.channels_last)

    print(f"[INFO] Trainable params: {count_parameters(model):,}")

    # loss/optim
    criterion = AsymmetricLoss(
        gamma_pos=ASL_GAMMA_POS, gamma_neg=ASL_GAMMA_NEG, clip=ASL_CLIP, pos_weight=pos_weights
    )
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

    # CORRECT scheduler: steps only when optimizer updates
    updates_per_epoch = int(np.ceil(epoch_steps / ACCUM_STEPS))
    total_updates = EPOCHS * updates_per_epoch
    warmup_updates = int(0.08 * total_updates)

    def lr_lambda(u):
        if u < warmup_updates:
            return (u + 1) / max(1, warmup_updates)
        progress = (u - warmup_updates) / max(1, (total_updates - warmup_updates))
        return 0.5 * (1.0 + np.cos(np.pi * progress))

    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

    scaler = torch.amp.GradScaler("cuda", enabled=(USE_AMP and device.type == "cuda"))
    ema = ModelEMA(model, decay=EMA_DECAY)

    history = {"train_loss": [], "val_loss": [], "val_auc": []}
    best_val_auc = 0.0
    patience = 0

    for epoch in range(EPOCHS):
        train_metrics = train_epoch_mixup(
            model, train_loaders, criterion, optimizer, device,
            scaler=scaler if (USE_AMP and device.type == "cuda") else None,
            ema=ema, scheduler=scheduler,
            epoch_steps=epoch_steps
        )

        # validate with EMA
        ema.apply_to(model)
        val_metrics, _, _ = validate(model, val_loader, criterion, device, thresholds=None, tta=False)
        ema.restore(model)

        history["train_loss"].append(train_metrics["loss"])
        history["val_loss"].append(val_metrics["loss"])
        history["val_auc"].append(val_metrics["mAUC"])

        print(f"Epoch {epoch+1:02d} | TrainLoss {train_metrics['loss']:.4f} | "
              f"Val mAUC {val_metrics['mAUC']:.4f} | Mix {train_metrics['mixup_ratio']:.1%} | "
              f"LR {optimizer.param_groups[0]['lr']:.2e}")

        if val_metrics["mAUC"] > best_val_auc:
            best_val_auc = val_metrics["mAUC"]
            patience = 0
            torch.save({"model": model.state_dict(), "ema": ema.shadow, "best_val_auc": best_val_auc},
                       f"{SAVE_DIR}/best_model.pth")
            print(f"  ✓ Saved best (EMA val mAUC={best_val_auc:.4f})")
        else:
            patience += 1

        if patience >= EARLY_STOP_PATIENCE:
            print(f"[INFO] Early stopping at epoch {epoch+1}")
            break

    # load best
    ckpt = torch.load(f"{SAVE_DIR}/best_model.pth", map_location=device)
    model.load_state_dict(ckpt["model"])
    ema.shadow = ckpt["ema"]
    print(f"\n[INFO] Best Val mAUC (EMA): {ckpt['best_val_auc']:.4f}")

    # thresholds from val (EMA)
    ema.apply_to(model)
    _, val_probs, val_labels = validate(model, val_loader, criterion, device, thresholds=None, tta=False)
    thresholds = find_optimal_thresholds(val_labels, val_probs)
    ema.restore(model)

    # test (EMA + optional TTA)
    print(f"\n[INFO] Testing on {TEST_DOMAIN} (EMA{' + TTA' if USE_TTA_ON_TEST else ''})...")
    ema.apply_to(model)
    test_metrics, _, _ = validate(model, test_loader, criterion, device, thresholds=thresholds, tta=USE_TTA_ON_TEST)
    ema.restore(model)

    print("\n" + "=" * 80)
    print(f"TEST RESULTS - {TEST_DOMAIN} (RetinaEffCNN)")
    print("=" * 80)
    print(f"mAUC:     {test_metrics['mAUC']:.4f}")
    print(f"mAP:      {test_metrics['mAP']:.4f}")
    print(f"Macro F1: {test_metrics['macro_f1']:.4f}")
    print("=" * 80)

    plot_training_curves(history, SAVE_DIR)
    print(f"[INFO] Saved curves/results to: {SAVE_DIR}")



HARMONIZING DATASETS

Processing train split...
RFMiD_v2 train: Found 507, Skipped 2

Processing val split...

Processing test split...
RFMiD_v2 test: Found 170, Skipped 4
Mode: FAST | IMG=288 | BATCH=32 | ACCUM=1 | AMP=True
Train: ODIR + RFMiD_v2 | Test: RFMiD_v1
[INFO] Domain 0: Loaded 7000 images from ODIR_train.csv
[INFO] Domain 1: Loaded 507 images from RFMiD_v2_train.csv
[INFO] Domain 0: Loaded 1000 images from ODIR_val.csv
[INFO] Domain 1: Loaded 177 images from RFMiD_v2_val.csv
[INFO] Domain 999: Loaded 640 images from RFMiD_v1_test.csv

[INFO] Positive class weights:
  N: 2.08
  D: 2.27
  G: 15.65
  C: 15.68
  A: 21.54
  H: 26.80
  M: 19.02
  O: 2.39
[INFO] epoch_steps (auto) = 218
[INFO] Trainable params: 2,015,618


Train:   0%|          | 0/218 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 01 | TrainLoss 0.9760 | Val mAUC 0.4867 | Mix 52.8% | LR 3.59e-04
  ✓ Saved best (EMA val mAUC=0.4867)


Epoch 02 | TrainLoss 0.7985 | Val mAUC 0.5162 | Mix 53.2% | LR 7.16e-04
  ✓ Saved best (EMA val mAUC=0.5162)


Epoch 03 | TrainLoss 0.6887 | Val mAUC 0.5196 | Mix 56.4% | LR 1.00e-03
  ✓ Saved best (EMA val mAUC=0.5196)


...

Epoch 33 | TrainLoss 0.3666 | Val mAUC 0.7225 | Mix 56.9% | LR 9.49e-06


Epoch 34 | TrainLoss 0.4096 | Val mAUC 0.7502 | Mix 49.5% | LR 2.38e-06
  ✓ Saved best (EMA val mAUC=0.7502)


Epoch 35 | TrainLoss 0.3928 | Val mAUC 0.7575 | Mix 47.7% | LR 0.00e+00
  ✓ Saved best (EMA val mAUC=0.7575)

[INFO] Best Val mAUC (EMA): 0.7575



[INFO] Testing on RFMiD_v1 (EMA)...



TEST RESULTS - RFMiD_v1 (RetinaEffCNN)
mAUC:     0.7433
mAP:      0.3917
Macro F1: 0.3261
[INFO] Saved curves/results to: ./results_lodo_mixup/retinaeffcnn_fast
